# =============================================================================
# AUTOML BASELINES: BAYESIAN OPTIMIZATION & EVOLUTIONARY STRATEGIES
# =============================================================================
## Purpose:
    - Implement State-of-the-art (SOTA) AutoML methods to compare with RL Agent.
    - Method 1: TPE (Bayesian Optimization) - Represents "Smart Search".
    - Method 2: CMA-ES (Evolutionary Strategy) - Represents "Evolutionary/Genetics".
    - This fulfills the Judge's requirement for "comparison with other methods".
# =============================================================================

# === 1. Install & Import ===

In [1]:
# Use if run on Kaggle
!rm -rf Sustainable_AI_Agent_Project
!git clone https://github.com/trongjhuongwr/Sustainable_AI_Agent_Project.git
%cd Sustainable_AI_Agent_Project

Cloning into 'Sustainable_AI_Agent_Project'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 68 (delta 26), reused 54 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 1.16 MiB | 15.43 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/kaggle/working/Sustainable_AI_Agent_Project


In [2]:
!pip install -q optuna cmaes

In [3]:
import os
import json
import copy
import warnings
import numpy as np
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import optuna
from optuna.samplers import TPESampler, CmaEsSampler
import random

warnings.filterwarnings("ignore")

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


# === 2. Configuration (Must Match RL Config) ===

In [4]:
class Config:
    PROCESSED_DATA_PATH = '/kaggle/input/baseline-model-saa/processed_data.pt'
    BASELINE_MODEL_PATH = '/kaggle/input/baseline-model-saa/baseline_model.pth'
    AUTOML_SAVE_PATH = '/kaggle/working/automl_results.json'
    
    SEQUENCE_LENGTH = 30
    INPUT_DIM = 4
    HIDDEN_DIM = 256
    N_LAYERS = 2
    OUTPUT_DIM = 1
    
    # AutoML Settings
    N_TRIALS = 50  # Same budget as RL Agent (approx equivalent to timesteps)
    SEED = 42
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False

seed_everything(Config.SEED)

# === 3. Model & Utility Functions (Reused) ===

In [5]:
class WeatherGRU(nn.Module):
    def __init__(self):
        super(WeatherGRU, self).__init__()
        self.gru = nn.GRU(Config.INPUT_DIM, Config.HIDDEN_DIM, Config.N_LAYERS, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(Config.HIDDEN_DIM, Config.OUTPUT_DIM)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return self.sigmoid(out)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def apply_optimizations(model, pruning_rate, quantization):
    # Pruning
    if pruning_rate > 0:
        model_copy = copy.deepcopy(model)
        
        for module in model_copy.modules():
            if isinstance(module, nn.Linear):
                prune.l1_unstructured(module, name='weight', amount=pruning_rate)
                prune.remove(module, 'weight')
        
        for module in model_copy.modules():
            if isinstance(module, nn.GRU):
                for name_param, param in list(module.named_parameters()):
                    if 'weight' in name_param:
                        prune.l1_unstructured(module, name=name_param, amount=pruning_rate)
                        prune.remove(module, name=name_param)
        
    else:
        model_copy = copy.deepcopy(model)
    
    # Quantization (CPU only)
    if quantization:
        model_copy.to('cpu')
        model_copy.eval()
        model_copy = torch.quantization.quantize_dynamic(
            model_copy, {nn.Linear, nn.GRU}, dtype=torch.qint8
        )
    return model_copy

# === 4. Objective Function (The "Reward" Logic) ===

In [6]:
def objective(trial, baseline_model, val_loader, baseline_metrics):
    """
    Optuna Objective Function.
    Maps (Pruning Rate, Quantization) -> Score.
    Score calculation MUST match the RL Reward function logic for fair comparison.
    """
    # 1. Suggest Hyperparameters
    pruning_rate = trial.suggest_float("pruning_rate", 0.0, 0.7, step=0.1)
    quantization = trial.suggest_categorical("quantization", [True, False])
    
    # 2. Apply Optimization
    model = apply_optimizations(baseline_model, pruning_rate, quantization)
    
    # 3. Evaluate (Fast evaluation on Validation Set)
    # Determine device based on quantization
    is_quantized = quantization
    device = torch.device("cpu") if is_quantized else Config.DEVICE
    model.to(device)
    model.eval()
    
    y_true, y_pred = [], []
    with torch.no_grad():
        for X, y in val_loader:
            X = X.to(device)
            preds = (model(X) > 0.5).float()
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            
    accuracy = accuracy_score(y_true, y_pred)
    params = count_parameters(model) # Proxy for Size/Energy
    
    # 4. Calculate Score (Equivalent to RL Reward)
    # Reward = (Acc_Delta * Scale) + (Param_Reduction * Scale) + (FLOPs_Reduction * Scale)
    
    # Reconstruct Metrics
    acc_drop = accuracy - baseline_metrics['accuracy']
    
    # Heuristic for FLOPs/Params
    param_ratio = params / baseline_metrics['params']
    quant_factor = 0.25 if is_quantized else 1.0
    
    # Reduction ratios (Higher is better)
    params_reduction = 1.0 - param_ratio
    flops_reduction = 1.0 - (param_ratio * quant_factor)
    
    # Penalties/Rewards (Must match RL Config exactly)
    ACCURACY_PENALTY_THRESHOLD = 0.98
    
    if accuracy < (baseline_metrics['accuracy'] * ACCURACY_PENALTY_THRESHOLD):
        score = -10.0 # Heavy penalty
    else:
        # Weighted Sum (Using same weights as RL)
        score = (acc_drop * 20.0) + (flops_reduction * 2.0) + (params_reduction * 1.0)
        
    return score

In [7]:
# === 5. Main Execution ===

# Load Data
processed_data = torch.load(Config.PROCESSED_DATA_PATH)
val_dataset = TensorDataset(processed_data['X_val'], processed_data['y_val'])
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

# Load Baseline
baseline_model = WeatherGRU()
baseline_model.load_state_dict(torch.load(Config.BASELINE_MODEL_PATH))
baseline_model.to(Config.DEVICE)

# Calculate Baseline Metrics
baseline_model.eval()
y_true_b, y_pred_b = [], []
with torch.no_grad():
    for X, y in val_loader:
        X = X.to(Config.DEVICE)
        preds = (baseline_model(X) > 0.5).float()
        y_true_b.extend(y.cpu().numpy())
        y_pred_b.extend(preds.cpu().numpy())
baseline_acc = accuracy_score(y_true_b, y_pred_b)
baseline_params = count_parameters(baseline_model)
baseline_metrics = {'accuracy': baseline_acc, 'params': baseline_params}

print(f"Baseline Metrics: {baseline_metrics}")

results = {}

Baseline Metrics: {'accuracy': 0.6608695652173913, 'params': 596225}


In [8]:
# --- Method 1: TPE (Bayesian Optimization) ---
print("\n--- Running Method 1: TPE (Bayesian Optimization) ---")
study_tpe = optuna.create_study(direction="maximize", sampler=TPESampler(seed=Config.SEED))
study_tpe.optimize(lambda trial: objective(trial, baseline_model, val_loader, baseline_metrics), n_trials=Config.N_TRIALS)

print(f"TPE Best Params: {study_tpe.best_params}")
print(f"TPE Best Score: {study_tpe.best_value}")
results['TPE'] = study_tpe.best_params

[I 2025-12-23 06:10:53,939] A new study created in memory with name: no-name-e0d3b4b3-fe75-403d-afa9-25d8422e0bb6



--- Running Method 1: TPE (Bayesian Optimization) ---


[I 2025-12-23 06:10:54,257] Trial 0 finished with value: 3.173913043478262 and parameters: {'pruning_rate': 0.2, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:54,317] Trial 1 finished with value: 2.82608695652174 and parameters: {'pruning_rate': 0.4, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:54,373] Trial 2 finished with value: 3.0 and parameters: {'pruning_rate': 0.0, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:54,384] Trial 3 finished with value: -10.0 and parameters: {'pruning_rate': 0.5, 'quantization': False}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:54,444] Trial 4 finished with value: 3.0 and parameters: {'pruning_rate': 0.6000000000000001, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:54,456] Trial 5 finished with value: 0.17391304347826209 and parameters: {'pruning_rate': 

TPE Best Params: {'pruning_rate': 0.30000000000000004, 'quantization': True}
TPE Best Score: 3.347826086956522


In [9]:
# --- Method 2: CMA-ES (Evolutionary Strategy) ---
print("\n--- Running Method 2: CMA-ES (Evolutionary Strategy) ---")
# CMA-ES works best with continuous variables, so it treats categorical as integer/float internally
study_cma = optuna.create_study(direction="maximize", sampler=CmaEsSampler(seed=Config.SEED))
study_cma.optimize(lambda trial: objective(trial, baseline_model, val_loader, baseline_metrics), n_trials=Config.N_TRIALS)

print(f"CMA-ES Best Params: {study_cma.best_params}")
print(f"CMA-ES Best Score: {study_cma.best_value}")
results['CMA-ES'] = study_cma.best_params

[I 2025-12-23 06:10:56,893] A new study created in memory with name: no-name-c2db2fab-fded-4ef3-9b51-baed5a133081
[I 2025-12-23 06:10:56,963] Trial 0 finished with value: 3.173913043478262 and parameters: {'pruning_rate': 0.2, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:56,964] `CmaEsSampler` only supports two or more dimensional continuous search space. `RandomSampler` is used instead of `CmaEsSampler`.
[I 2025-12-23 06:10:57,029] Trial 1 finished with value: 2.82608695652174 and parameters: {'pruning_rate': 0.4, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:57,030] `CmaEsSampler` only supports two or more dimensional continuous search space. `RandomSampler` is used instead of `CmaEsSampler`.



--- Running Method 2: CMA-ES (Evolutionary Strategy) ---


[I 2025-12-23 06:10:57,092] Trial 2 finished with value: 3.0 and parameters: {'pruning_rate': 0.0, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:57,093] `CmaEsSampler` only supports two or more dimensional continuous search space. `RandomSampler` is used instead of `CmaEsSampler`.
[I 2025-12-23 06:10:57,104] Trial 3 finished with value: -10.0 and parameters: {'pruning_rate': 0.5, 'quantization': False}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:57,105] `CmaEsSampler` only supports two or more dimensional continuous search space. `RandomSampler` is used instead of `CmaEsSampler`.
[I 2025-12-23 06:10:57,163] Trial 4 finished with value: 3.0 and parameters: {'pruning_rate': 0.6000000000000001, 'quantization': True}. Best is trial 0 with value: 3.173913043478262.
[I 2025-12-23 06:10:57,164] `CmaEsSampler` only supports two or more dimensional continuous search space. `RandomSampler` is used instead of `CmaEsSampler`.
[I 2

CMA-ES Best Params: {'pruning_rate': 0.30000000000000004, 'quantization': True}
CMA-ES Best Score: 3.347826086956522


In [10]:
# Save Results
with open(Config.AUTOML_SAVE_PATH, 'w') as f:
    json.dump(results, f)
print(f"\nAutoML results saved to {Config.AUTOML_SAVE_PATH}")


AutoML results saved to /kaggle/working/automl_results.json
